#모델 설명


1. 개요
  
  1-1. 사전 학습 모델(RoBERTa)를 이용하여 발화의 종류를 분류

  1-2. 프레임워크

  전처리 > 모델 학습 (RoBERTa-small, RoBERTa-large[smote 적용/미적용]) > 앙상블(우선순위에 따라 선택) 


2. 전처리 사항

  2-1. 띄어쓰기(PyKoSpacing), 맞춤법 교정(py-hanspell)
    - 제공 데이터에 적용하여 업로드 후 학습 데이터로 사용
  
  2-2. 오버샘플링(SMOTE)
    - 소수 클래스(2~50건) 데이터에 대해 SMOTE 오버 샘플링 기법을 적용하여 클래스 불균형 완화
    - 텍스트 데이터를 TfidfVectorizer로 벡터화 시켜 SMOTE를 적용
    - RoBERTa-large 모델 두 가지 중 하나에는 미적용
    

3. 모델 학습 및 앙상블

  3-1. 3가지 모델을 활용, 제공된 1만건의 데이터를 학습 (27개 클래스)

  3-2. 모델 앙상블하여 최종 결과 도출
  - 3가지 모델 다수결 투표
  - 결과가 모두 다른 경우, 제공된 우선순위 따라 도출

*전처리 등의 기능을 담은 Class, def 등의 코드는 .py 파일로 저장 후 import 하여 사용

- model 폴더 내 preprocess 폴더에 저장
- Class_Roberta : RoBERTa용 데이터 세트 로딩
- Preprocessor : 레이블 인코딩, 리스트 형태 변환, 테스트 데이터 전처리, 오버샘플링


*모델 및 패키지 출처 : 
- github.com/pytorch/fairseq/tree/master/examples/roberta
- github.com/haven-jeon/PyKoSpacing
- github.com/ssut/py-hanspell

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Class 정보를 불러오기 위해 필요
import sys
sys.path.append(Model_PATH)
print(sys.path)

#공용함수

In [ ]:
# 수정금지: 타임스탬프용 함수
from datetime import datetime
def printt(*args,**kwargs):
  now = datetime.now()
  now_str = "{:02}:{:02}:{:02}".format(now.hour,now.minute,now.second)
  print(now_str, *args,**kwargs)
  return int(now.hour)*60*60+int(now.minute)*60+int(now.second)

#연관 패키지 설치 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install adabelief-pytorch
!pip install AdamP

#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.

import os
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tqdm.auto import tqdm

# Torch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# NLP
import gluonnlp as nlp


# Transformer
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification

# Optimizer
from transformers import AdamW
from adamp import AdamP
from adabelief_pytorch import AdaBelief

# Oversampling 
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

# 띄어쓰기 및 맞춤법 교정
from pykospacing import Spacing
from py_hanspell_master.py_hanspell_master.hanspell import spell_checker

# .py files
import preprocess.Preprocessor as preprocessor 
import preprocess.Class_Roberta as class_roberta 


os.environ['CUDA_LAUNCH_BLOKING'] = '1'
device = torch.device('cuda:0')

# 파일로딩 (다른작업 금지)

### 토크나이저 및 모델 로드
- Kobert, RoBERTa의 토크나이저 및 모델 로드

In [ ]:
# RoBERTa 토크나이저 및 모델 로드
tokenizer_roberta_large = AutoTokenizer.from_pretrained(PATH + '/tokenizer_roberta_large')
tokenizer_roberta_small = AutoTokenizer.from_pretrained(PATH + '/tokenizer_roberta_small')
model_roberta_large_1 = torch.load(PATH + '/model_roberta_large.pt')
model_roberta_large_2 = torch.load(PATH + '/model_roberta_large.pt')
model_roberta_small = torch.load(PATH + '/model_roberta_small.pt')

## 파일 로드
- 학습 데이터, 테스트 데이터 로딩 및 전처리

In [ ]:
PATH_2 = DATA_PATH

In [ ]:
train_PATH = PATH_2 + '/한스펠트레인_인덱스.xlsx'
num_PATH = PATH_2 + '/통합.xlsx'
test_PATH = PATH_2 + '/input_data.xlsx'

preprocessing = preprocessor.Preprocessor() # 로딩용 .py 파일

# 학습용 데이터
df,encoder = preprocessing.preprocessing_4_train(train_PATH) # 띄어쓰기 및 맞춤법 교정이 적용된 학습 데이터
numb_df = pd.read_excel(num_PATH, sheet_name='우선순위') # 원본 데이터 내의 우선 순위 관련 데이터 로딩

# 테스트 데이터
test_raw = pd.read_excel(test_PATH)

Preprocessor is Operating


# 모델 학습

In [ ]:
# 수정금지: 타임스탬프
_model_build_start_time = printt("Model building: Start")
_model_build_start_time

08:24:15 Model building: Start


30255

In [ ]:
#TODO: 블럭에 모델 학습 - 빌딩 코드를 넣으세요. (시간측정 구간)

### 모델 별 적용 파라미터

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)

# 공용 파라미터
max_grad_norm = 1
log_interval = 500
warmup_ratio = 0.5

## Roberta-large 파라미터
BATCH_SIZE = 16
NUM_EPOCHS = 5
L_RATE = 1e-5
MAX_LEN = 30
NUM_CORES = 5

## Roberta-small 파라미터
NUM_EPOCHS_S = 10
L_RATE_S = 1e-3

### 전처리 및 입력 데이터로 변환

In [ ]:
# 한글 영문만 남기고 모두 제거하는 코드
def preprocessing(text):
    # 특수문자나 이모티콘 등 아래의 특수기호들을 제거합니다(%등은 남김).
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 위에서 특수문자를 제거한 text에서 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

# 띄어쓰기 및 맞춤법 교정
def preprocessing_2 (df):
  for i in tqdm(range(len(df))):
      df.loc[i,'발화'] = spacing(df.loc[i,'발화'])
  for i in tqdm(range(len(df))):
      sent = df.loc[i,'발화']
      spelled_sent = spell_checker.check(sent)
      hanspell_sent = spelled_sent.checked
      df.loc[i,'발화'] = hanspell_sent

In [ ]:
# 학습용 데이터 전처리(오버 샘플링)
df_over = preprocessor.oversampling_4_roberta(df)

# 테스트 데이터 전처리
test = preprocessor.test_prepro(test_raw)
test['발화']= test['발화'].apply(lambda x: preprocessing(x))
spacing = Spacing()
preprocessing_2(test)


## Roberta용 데아터 세트

train_data_large_1 = class_roberta.TrainDataset_large(df_over, tokenizer_roberta_large,MAX_LEN)
test_data_large_1 = class_roberta.TestDataset_large(test, tokenizer_roberta_large,MAX_LEN)
train_data_large_2 = class_roberta.TrainDataset_large(df, tokenizer_roberta_large,MAX_LEN)
test_data_large_2 = class_roberta.TestDataset_large(test, tokenizer_roberta_large,MAX_LEN)
train_data_small = class_roberta.TrainDataset_small(df_over, tokenizer_roberta_small,MAX_LEN)
test_data_small = class_roberta.TestDataset_small(test, tokenizer_roberta_small,MAX_LEN)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# Training Model

In [ ]:
# 학습 정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## RoBerta-large_1
- 오버샘플링을 적용한 데이터를 학습시킨 모델

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 5
L_RATE = 1e-5
MAX_LEN = 30
NUM_CORES = 5

In [ ]:
# DataLoader를 적용하여 모델 입력 데이터로 변환
train_dataloader_4_roberta_large_1 = torch.utils.data.DataLoader(train_data_large_1,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader_4_roberta_large_1 = torch.utils.data.DataLoader(test_data_large_1,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)

# 층 파라미터 조정
n=0
for name, child in model_roberta_large_1.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=328: 
          param.requires_grad = False
        h+=1
    n+=1


# Optimizer 및 learning 스케줄 조정
model_roberta_large_1.to(device)
optimizer = AdaBelief(model_roberta_large_1.parameters(), lr=L_RATE, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)
t_total = len(train_dataloader_4_roberta_large_1) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


In [ ]:
# RoBERTa-large 모델 학습
for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model_roberta_large_1.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm(train_dataloader_4_roberta_large_1)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model_roberta_large_1(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss =outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_roberta_large_1.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), 
                                                                     train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.4931623935699463 train acc 0.0
epoch 1 batch id 501 loss 1.0941153764724731 train acc 0.4024451097804391
epoch 1 train acc 0.4947033898305085


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.41902369260787964 train acc 0.875
epoch 2 batch id 501 loss 0.7442509531974792 train acc 0.8638972055888223
epoch 2 train acc 0.8713084232152029


...

epoch 3 batch id 1 loss 0.38086554408073425 train acc 0.875
epoch 3 batch id 501 loss 0.1891448050737381 train acc 0.9292664670658682
epoch 3 train acc 0.9295497346344804


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.050560325384140015 train acc 1.0
epoch 4 batch id 501 loss 0.3518050014972687 train acc 0.9529690618762475
epoch 4 train acc 0.9537964389659306


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.04112333431839943 train acc 1.0
epoch 5 batch id 501 loss 0.03165051341056824 train acc 0.9715568862275449
epoch 5 train acc 0.9704352850539292


## RoBerta-large_2
- 오버샘플링을 적용하지 않은 데이터를 학습시킨 모델

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 5
L_RATE = 1e-4
MAX_LEN = 40
NUM_CORES = 5

In [ ]:
# DataLoader를 적용하여 모델 입력 데이터로 변환
train_dataloader_4_roberta_large_2 = torch.utils.data.DataLoader(train_data_large_2,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader_4_roberta_large_2 = torch.utils.data.DataLoader(test_data_large_2,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)

# 층 파라미터 조정
n=0
for name, child in model_roberta_large_2.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=328: 
          param.requires_grad = False
        h+=1
    n+=1


# Optimizer 및 learning 스케줄 조정
model_roberta_large_2.to(device)
optimizer = AdaBelief(model_roberta_large_2.parameters(), lr=L_RATE, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)
t_total = len(train_dataloader_4_roberta_large_2) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


In [ ]:
# RoBERTa-large 모델 학습
for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model_roberta_large_2.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm(train_dataloader_4_roberta_large_2)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model_roberta_large_2(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss =outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_roberta_large_2.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), 
                                                                     train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.4933698177337646 train acc 0.0625
epoch 1 batch id 501 loss 0.0997375100851059 train acc 0.7173153692614771
epoch 1 train acc 0.7528


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.46607327461242676 train acc 0.875
epoch 2 batch id 501 loss 0.23707368969917297 train acc 0.9206586826347305
epoch 2 train acc 0.9201


...

epoch 3 batch id 1 loss 0.1842116415500641 train acc 0.9375
epoch 3 batch id 501 loss 0.24514199793338776 train acc 0.9320109780439122
epoch 3 train acc 0.9313


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.4581618010997772 train acc 0.9375
epoch 4 batch id 501 loss 0.06360575556755066 train acc 0.9578343313373253
epoch 4 train acc 0.9592


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.042984459549188614 train acc 1.0
epoch 5 batch id 501 loss 0.003027993720024824 train acc 0.9835329341317365
epoch 5 train acc 0.9843


### RoBERTa-small

In [ ]:
BATCH_SIZE = 16
MAX_LEN = 30
NUM_CORES = 5
## Roberta-small 파라미터
NUM_EPOCHS_S = 10
L_RATE_S = 1e-3

In [ ]:
# DataLoader를 적용하여 모델 입력 데이터로 변환
train_dataloader_4_roberta_small = torch.utils.data.DataLoader(train_data_small,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader_4_roberta_small = torch.utils.data.DataLoader(test_data_small,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)


# 층 파라미터 조정
n=0
for name, child in model_roberta_small.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=312: 
          param.requires_grad = False
        h+=1
    n+=1


# Optimizer 및 learning 스케줄 조정
model_roberta_small.to(device)
optimizer = AdaBelief(model_roberta_small.parameters(), lr=L_RATE_S, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)
t_total = len(train_dataloader_4_roberta_small) * NUM_EPOCHS_S
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# RoBERTa-small 모델 학습
for e in range(NUM_EPOCHS_S):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model_roberta_small.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm(train_dataloader_4_roberta_small)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model_roberta_small(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss =outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_roberta_small.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), 
                                                                     train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.28810715675354 train acc 0.0625
epoch 1 batch id 501 loss 1.0349183082580566 train acc 0.4948852295409182
epoch 1 train acc 0.5537044170518747


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.7314167618751526 train acc 0.6875
epoch 2 batch id 501 loss 0.1846245378255844 train acc 0.8063872255489022
epoch 2 train acc 0.813334617360041


...

epoch 7 batch id 1 loss 0.38151034712791443 train acc 0.875
epoch 7 batch id 501 loss 0.46812203526496887 train acc 0.8705089820359282
epoch 7 train acc 0.8697461907207671


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 8 batch id 1 loss 0.27677255868911743 train acc 0.875
epoch 8 batch id 501 loss 0.23745422065258026 train acc 0.8799900199600799
epoch 8 train acc 0.8807460195172059


  0%|          | 0/649 [00:00<?, ?it/s]

epoch 9 batch id 1 loss 0.02475973404943943 train acc 1.0


In [ ]:
# 수정금지: 타임스탬프
_model_build_end_time = printt("Model building: Start")
print(_model_build_end_time - _model_build_start_time)

#모델 분류

In [ ]:
# 수정금지: 타임스탬프
_test_start_time = printt("TEST: Start")

## 앙상블을 통한 최종 결과 도출
- 각 모델 별 결과값을 도출하여 다수결 투표 진행 후 최종 결과 선정
- 3개의 모델이 모두 다른 값을 가질 경우 우선 순위를 반영하여 최종 결과 선정

### 모델 별 TEST 데이터 분류

In [ ]:
#TODO: 해당 블럭에 테스트 수행을 위한 코드를 넣으세요. (시간측정 구간)
#분석 파일은 tsv 파일로 제공되며, 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 =공백)으로 제공됩니다.


# RoBERTa-large_1 모델 테스트 파일 분류 수행
preds_reberta_1 = [] 
model_roberta_large_1.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm(test_dataloader_4_roberta_large_1)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model_roberta_large_1(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds_reberta_1.append(inp.detach().cpu().numpy())
Preds_reberta_1 = np.array(preds_reberta_1)

# RoBERTa-large_2 모델 테스트 파일 분류 수행
preds_reberta_2 = [] 
model_roberta_large_2.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm(test_dataloader_4_roberta_large_2)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model_roberta_large_2(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds_reberta_2.append(inp.detach().cpu().numpy())
Preds_reberta_2 = np.array(preds_reberta_2)


# RoBERTa-small 모델 테스트 파일 분류 수행
preds_reberta_s = [] 
model_roberta_small.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm(test_dataloader_4_roberta_small)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model_roberta_small(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds_reberta_s.append(inp.detach().cpu().numpy())
Preds_reberta_s = np.array(preds_reberta_s)

# 하위 리스트 제거
del preds_reberta_1, preds_reberta_2, preds_reberta_s

In [ ]:
# 결과값을 우선 순위로 변환하는 함수
def inverse (col_name, val_list):
  val_list3= np.argmax(val_list, axis=1)
  test[col_name] = encoder.inverse_transform(val_list3)
  for j in tqdm(range(len(numb_df))):
    test.loc[(test[col_name] == numb_df['의도명(유형)'][j]), col_name+"_la"] = numb_df['우선순위'][j]
  return test.head()

# 중복 체크 함수
def not_duplicates(seq):
    return len(seq) == len(set(seq))

# 우선 순위 변환 함수 적용
inverse('robert_L_1',Preds_reberta_1)
inverse('robert_L_2',Preds_reberta_2)
inverse('robert_S',Preds_reberta_s)

# 모델 별 결과값 투표 후 다수결에 따라 최종 결과값 도출
# 3개의 모델의 결과값이 다를 경우 우선 순위를 반영하여 결과값 도출

val_list = []
for i in tqdm(range(len(test))):
  x = test.loc[i,['robert_L_1_la','robert_L_2_la','robert_S_la']]
  if not_duplicates(x): # 중복 값이 없을 경우 우선 순위 적용
    y = min(x)
    val_list.append(int(y))
  else: # 중복이 있을 경우 가장 많이 선정된 값을 최종 결과값으로 선정
    z = x.value_counts().index[0]
    val_list.append(int(z))
test['pred'] = val_list


# 도출된 숫자형 결과값을 기존 텍스트로 변경
for j in tqdm(range(len(numb_df))):
  test.loc[(test['pred'] == numb_df['우선순위'][j]), "INT"] = numb_df['의도명(유형)'][j]


# Test 파일의 INT를 결과값으로 수정
test_raw['INT'] = test['INT']

In [ ]:
# 수정금지: 타임스탬프
_test_end_time = printt("Model building: Start")
print(_test_end_time - _test_start_time)

# 결과출력

In [ ]:
#TODO:해당 블럭에 테스트 결과를 파일로 저장하는 코드를 넣으세요. (시간측정 제외)
#저장 파일은tsv 파일로 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 = 테스트 결과 도출된 양식)으로 저장

test_raw.to_excel(Result_PATH,index=False)